In [34]:
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from helpers import llm
from langchain.tools import tool

llm = llm()


@tool
def test_tool(question: str):
    "This tool provides information about Fritz Karuugaa. Give a question as input."
    return "Fritz Karuugaa is a software developer"


tools = [test_tool]

llm_with_tools = llm.bind_tools(tools)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(
            "Du bist ein hilfsbereiter Assistent. Löse deine Aufgaben auf eine humorvolle Art und weiße. Antworte zusätzlich immer mit einem Witz.",
        ),
        HumanMessagePromptTemplate.from_template("{input}"),
        MessagesPlaceholder("intermediate_steps"),
    ]
)

chain = prompt | llm_with_tools

In [35]:
import operator
from typing import Annotated, TypedDict

from langchain_core.agents import AgentActionMessageLog, AgentFinish
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableLambda
from langgraph.graph import END, StateGraph


class AgentState(TypedDict):
    input: str
    intermediate_steps: Annotated[list[tuple[AgentActionMessageLog, str]], operator.add]
    answer: str


def should_continue(state):
    return "continue" if state["intermediate_steps"][-1].tool_calls else "end"


def call_model(state, config):
    response = chain.invoke(state, config=config)
    print("state", state)
    if isinstance(response, AgentFinish):
        return {"answer": response.answer}
    else:
        return {"intermediate_steps": [response]}
    # return {"intermediate_steps": [chain.invoke(state, config=config)]}


def _invoke_tool(tool_call):
    tool = {tool.name: tool for tool in tools}[tool_call["name"]]
    return ToolMessage(tool.invoke(tool_call["args"]), tool_call_id=tool_call["id"])


tool_executor = RunnableLambda(_invoke_tool)


def call_tools(state):
    last_message = state["intermediate_steps"][-1]
    return {"intermediate_steps": tool_executor.batch(last_message.tool_calls)}


workflow = StateGraph(AgentState)
workflow.add_node("agent", call_model)
workflow.add_node("action", call_tools)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END,
    },
)
workflow.add_edge("action", "agent")
graph = workflow.compile()
graph.invoke({"input": "Was weißt du über Fritz Karuugaa?"})

state {'input': 'Was weißt du über Fritz Karuugaa?', 'intermediate_steps': [], 'answer': None}
state {'input': 'Was weißt du über Fritz Karuugaa?', 'intermediate_steps': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yAWGsw9xWdEAuODTXsGZ8Ugk', 'function': {'arguments': '{"question":"Tell me about Fritz Karuugaa"}', 'name': 'test_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 110, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-88015fa4-1020-4c7d-b162-232a9cd9205e-0', tool_calls=[{'name': 'test_tool', 'args': {'question': 'Tell me about Fritz Karuugaa'}, 'id': 'call_yAWGsw9xWdEAuODTXsGZ8Ugk'}]), ToolMessage(content='Fritz Karuugaa is a software developer', tool_call_id='call_yAWGsw9xWdEAuODTXsGZ8Ugk')], 'answer': None}


{'input': 'Was weißt du über Fritz Karuugaa?',
 'intermediate_steps': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yAWGsw9xWdEAuODTXsGZ8Ugk', 'function': {'arguments': '{"question":"Tell me about Fritz Karuugaa"}', 'name': 'test_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 110, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-88015fa4-1020-4c7d-b162-232a9cd9205e-0', tool_calls=[{'name': 'test_tool', 'args': {'question': 'Tell me about Fritz Karuugaa'}, 'id': 'call_yAWGsw9xWdEAuODTXsGZ8Ugk'}]),
  ToolMessage(content='Fritz Karuugaa is a software developer', tool_call_id='call_yAWGsw9xWdEAuODTXsGZ8Ugk'),
  AIMessage(content='Fritz Karuugaa is a software developer. Why did the computer go to the doctor? Because it had a virus!', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 149, 'tot